In [32]:
import (
	"fmt"
	"log"
	"os"
	"time"

	"github.com/evertoncolling/poc-requests-go/pkg/api"
	"github.com/evertoncolling/poc-requests-go/pkg/dto"

	grob "github.com/MetalBlueberry/go-plotly/graph_objects"
	"github.com/MetalBlueberry/go-plotly/offline"
	"github.com/joho/godotenv"
)

// Find a unit by external ID
func findUnitByExternalId(unitList *dto.UnitList, externalId string) (*dto.Unit, error) {
	for _, unit := range unitList.Items {
		if unit.ExternalId == externalId {
			return &unit, nil
		}
	}
	return nil, fmt.Errorf("unit not found with ExternalId: %s", externalId)
}

func getClient() api.CogniteClient {
    // Load environment variables
    err := godotenv.Load(".env")
    if err != nil {
        log.Fatalf("Error loading .env file")
    }

    // Read client credentials from environment variables
    clientID := os.Getenv("CLIENT_ID")
    clientSecret := os.Getenv("CLIENT_SECRET")
    tenantID := os.Getenv("TENANT_ID")
    cluster := os.Getenv("CDF_CLUSTER")
    project := os.Getenv("CDF_PROJECT")

    credentials := api.AzureADClientCredentials(
        clientID,
        clientSecret,
        tenantID,
        cluster,
    )
    clientConfig := api.ClientConfig{
        ClientName:  "poc-requests-go",
        Cluster:     cluster,
        Project:     project,
        Credentials: credentials,
    }
    client := api.NewCogniteClient(clientConfig)
    return client
}

// initialize client
var client = getClient()

%%
fmt.Println("### Testing fetching some time series")

// List time series
tsList, err := client.TimeSeries.List(100, false, "", "", nil, nil, "")
if err != nil {
	fmt.Println("Error:", err)
	return
}
fmt.Println("Time Series Count:", len(tsList.Items))

### Testing fetching some time series
Time Series Count: 60


In [15]:
%%
// Filter time series
filter := &dto.TimeSeriesFilter{
	UnitQuantity: "Pressure",
}
filteredTsList, err := client.TimeSeries.Filter(filter, nil, 100, "", "", nil)
if err != nil {
	fmt.Println("Error:", err)
	return
}

fmt.Println("Filtered Time Series Count:", len(filteredTsList.Items))

Filtered Time Series Count: 3


In [34]:
func getUnitList() dto.UnitList {
	unitList, err := client.Units.List()
	if err != nil {
		fmt.Println("Error:", err)
		return dto.UnitList{}
	}
	return unitList
}
var unitList = getUnitList()

%%
fmt.Println("### Testing fetching the Unit catalog")
fmt.Println("Unit Count:", len(unitList.Items))

### Testing fetching the Unit catalog
Unit Count: 276


In [37]:
%%
// Fetch the latest data points
fmt.Println("### Latest Data Points")
externalIds := []string{"EVE-TI-FORNEBU-01-2", "EVE-TI-FORNEBU-01-3"}
var latestDataPointsQueryItems []dto.LatestDataPointsQueryItem
for _, externalId := range externalIds {
	latestDataPointsQueryItems = append(latestDataPointsQueryItems, dto.LatestDataPointsQueryItem{
		ExternalId: externalId,
	})
}
latestDataPoints, err := client.TimeSeries.RetrieveLatest(
	&latestDataPointsQueryItems,
	nil,
)
if err != nil {
	fmt.Println("Error:", err)
	return
}
for _, latestDataPoint := range latestDataPoints.Items {
	unit, err := findUnitByExternalId(&unitList, latestDataPoint.UnitExternalId)
	if err != nil {
		fmt.Println("Error:", err)
		return
	}
	fmt.Println(latestDataPoint.ExternalId+" ["+unit.Symbol+"]"+":", latestDataPoint.DatapointType)
}

### Latest Data Points
EVE-TI-FORNEBU-01-2 [°C]: &{datapoints:{timestamp:1705600500000  value:-10.1}}
EVE-TI-FORNEBU-01-3 [°C]: &{datapoints:{timestamp:1705600203000  value:-15.78}}


In [40]:
%%
// Get many data points (performance test)
fmt.Println("### Data Points (performance test)")
items := []dto.DataPointsQueryItem{
	{
		ExternalId: "EVE-TI-FORNEBU-01-2",
		Start:      "300d-ago",
		End:        "now",
		Limit:      100000,
	},
}
start := time.Now()
dataPoints, err := client.TimeSeries.RetrieveData(
	&items,
	nil, nil, nil, nil, nil, nil, nil,
)
elapsed := time.Since(start)
if err != nil {
	fmt.Println("Error:", err)
	return
}
dps := dataPoints.Items[0]
fmt.Println("Data Points External ID:", dps.ExternalId)
unit, err := findUnitByExternalId(&unitList, dps.UnitExternalId)
if err != nil {
	fmt.Println("Error:", err)
	return
}
fmt.Println("Data Points Unit:", dps.UnitExternalId, " -> ["+unit.Symbol+"]")
// not sure if there is a simpler way to get to the data points :)
switch v := (*dps).DatapointType.(type) {
case *dto.DataPointListItem_NumericDatapoints:
	fmt.Println("Data Points Count:", len(v.NumericDatapoints.Datapoints))
case *dto.DataPointListItem_StringDatapoints:
	fmt.Println("Data Points Count:", len(v.StringDatapoints.Datapoints))
case *dto.DataPointListItem_AggregateDatapoints:
	fmt.Println("Data Points Count:", len(v.AggregateDatapoints.Datapoints))
default:
	fmt.Println("Unknown data point type:", v)
}
fmt.Printf("Time taken: %s\n", elapsed)

### Data Points (performance test)
Data Points External ID: EVE-TI-FORNEBU-01-2
Data Points Unit: temperature:deg_c  -> [°C]
Data Points Count: 71465
Time taken: 167.381375ms


In [46]:
%%
// let's try a more reasonable number of data points
fmt.Println("### Data Points (for plotting)")
items := []dto.DataPointsQueryItem{
	{
		ExternalId:  "EVE-TI-FORNEBU-01-2",
		Start:       "300d-ago",
		End:         "now",
		Aggregates:  []string{"average"},
		Granularity: "1h",
		Limit:       10000,
	},
}
start := time.Now()
dataPoints, err := client.TimeSeries.RetrieveData(
	&items,
	nil, nil, nil, nil, nil, nil, nil,
)
elapsed := time.Since(start)
if err != nil {
	fmt.Println("Error:", err)
	return
}

// Prepare the data for plotting
dps := dataPoints.Items[0]
aggregateDatapointsType := dps.DatapointType.(*dto.DataPointListItem_AggregateDatapoints)
aggregateDatapoints := aggregateDatapointsType.AggregateDatapoints.Datapoints
values := make([]float64, 0, len(aggregateDatapoints))
timestamps := make([]time.Time, 0, len(aggregateDatapoints))
for _, datapoint := range aggregateDatapoints {
	if datapoint != nil {
		values = append(values, datapoint.Average)
		timestamp := time.Unix(0, datapoint.Timestamp*int64(time.Millisecond))
		timestamps = append(timestamps, timestamp)
	}
}
fmt.Println("Data Points Count:", len(values))
fmt.Printf("Time taken: %s\n", elapsed)
unit, err := findUnitByExternalId(&unitList, dps.UnitExternalId)
if err != nil {
	fmt.Println("Error:", err)
	return
}

// Plot the data
fig := &grob.Fig{
	Data: grob.Traces{
		&grob.Bar{
			Type: grob.TraceTypeScatter,
			X:    timestamps,
			Y:    values,
			Name: dps.ExternalId,
		},
	},
	Layout: &grob.Layout{
		Title: &grob.LayoutTitle{
			Text: fmt.Sprintf("Fetched %d data points - hourly aggregates", len(values)),
			Font: &grob.LayoutTitleFont{
				Size: 24.0,
			},
		},
		Font: &grob.LayoutFont{
			Family: "Roboto",
			Size:   12,
			Color:  "black",
		},
		Xaxis: &grob.LayoutXaxis{
			Showspikes:     grob.True,
			Spikemode:      grob.LayoutXaxisSpikemode("across"),
			Spikethickness: 1.0,
			Spikedash:      "solid",
		},
		Yaxis: &grob.LayoutYaxis{
			Title: &grob.LayoutYaxisTitle{
				Text: fmt.Sprintf("%s [%s]", unit.Quantity, unit.Symbol),
			},
		},
		Spikedistance: -1,
		Legend: &grob.LayoutLegend{
			Orientation: grob.LayoutLegendOrientation("h"),
			Yanchor:     grob.LayoutLegendYanchor("bottom"),
			Y:           1.02,
			Xanchor:     grob.LayoutLegendXanchor("right"),
			X:           1.0,
		},
		Showlegend: grob.True,
		Height:     800,
	},
}

// for now we can only show the plot in a browser
offline.Show(fig)

### Data Points (for plotting)
Data Points Count: 6395
Time taken: 132.273458ms
